In [1]:
# if run on colab
!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=torc

In [1]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch
from torcheeg.trainers import ClassifierTrainer
import pytorch_lightning as pl

In [3]:
root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

In [2]:
root_data_path = './BCICIV-2a-mat/'

In [4]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    online_transform=transforms.Compose([
        transforms.To2d(),
        transforms.MinMaxNormalize(),
        transforms.ToTensor()
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-17 12:19:58] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-17 12:19:58] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|                                                                                                                       | 0/18 [00:00<?, ?it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 0it [00:00, ?it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 1it [00:00,  1.95it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 38it [00:00, 81.71it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 71it [00:00, 140.25it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 103it [00:00, 185.41it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 136it [00:00, 223.17it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 171it [00:01, 256.53it/s]
[RECORD ./BCICIV-2a-mat/A06E.mat]: 208it [00:01, 287.64it/s]
[RECORD ./BCICIV-2a-ma

In [5]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A06E_0        A06         0       E     A06    3   
1         2254    4004    A06E_1        A06         1       E     A06    3   
2         4172    5922    A06E_2        A06         2       E     A06    3   
3         6124    7874    A06E_3        A06         3       E     A06    3   
4         8132    9882    A06E_4        A06         4       E     A06    3   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A05T_283        A05        43       T     A05    8   
5180     88657   90407  A05T_284        A05        44       T     A05    8   
5181     90585   92335  A05T_285        A05        45       T     A05    8   
5182     92699   94449  A05T_286        A05        46       T     A05    8   
5183     94758   96508  A05T_287        A05        47       T     A05    8   

      label  _record_id  
0         1   _recor

In [6]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [7]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'


for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )

    model = EEGNet(
        chunk_size=7*250,
        num_electrodes=22,
        num_classes=4
    )
    
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/eegnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-17 12:20:23] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-17 12:20:23] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/quang/micromamba/envs/cognitive-computing/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder

Epoch 0: 100%|███████████████████████████████████████████████████████████████| 68/68 [00:18<00:00,  3.76it/s, v_num=0, train_loss=1.440, train_accuracy=0.188]

[2024-05-17 12:20:43] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.407 train_accuracy: 0.260 

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|███████████████████████████████████████████████████████████████| 68/68 [00:18<00:00,  3.76it/s, v_num=0, train_loss=1.440, train_accuracy=0.188]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/quang/Study/Cognitive-Science/cognitive-science-final-project/lightning_logs


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: examples_pipeline/eegnet_model/3/lightning_logs

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | EEGNet           | 4.9 K 
1 | ce_fn         | CrossEntropyLoss | 0     
2 | train_loss    | MeanMetric       | 0     
3 | val_loss      | MeanMetric       | 0     
4 | test_loss     | MeanMetric       | 0     
5 | train_metrics | MetricCollection | 0     
6 | val_metrics   | MetricCollection | 0     
7 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
4.9 K     Trainable params
0         Non-trainable params
4.9 K     Total params
0.020     Total estimated model params size (MB)


Epoch 0:  11%|██████▊                                                         | 8/75 [00:02<00:18,  3.57it/s, v_num=0, train_loss=1.430, train_accuracy=0.219]

/home/quang/micromamba/envs/cognitive-computing/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing DataLoader 0:   0%|                                                                                                             | 0/7 [00:00<?, ?it/s]

TypeError: 'NoneType' object is not subscriptable

In [8]:
!mkdir weights

In [9]:
eeg_weights_path = './weights/eeg_weights.pt'
torch.save(model.state_dict(), eeg_weights_path)
# to load weight:
# model.load_state_dict(torch.load(eeg_weights_path))